In [6]:
import pandas as pd
import numpy as np

from ipywidgets import interact, fixed
from sklearn.metrics import mean_squared_error

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.figure_factory as ff
init_notebook_mode(connected=True)

In [7]:
def round_thresh(x, t):
    if x % 1 > t:
        return np.ceil(x)
    else:
        return np.floor(x)

def rsme(y1, y2):
    return np.sqrt(mean_squared_error(y1, y2))

In [21]:
respostas = pd.read_csv('./results/validacao.csv', encoding = "ISO-8859-1")

In [67]:
erro_ano_passado = rsme(respostas.QUANTIDADE, respostas.LAST_YEAR)
erro_2_ano = rsme(respostas.QUANTIDADE, respostas.LAST_YEAR_2)
erro_media_dos_anos = rsme(respostas.QUANTIDADE, np.round(respostas.LAST_YEAR_MEAN))
erro_preds_all = rsme(respostas.QUANTIDADE, respostas.preds_all)
erro_preds_retrain = rsme(respostas.QUANTIDADE, np.round(respostas.preds_retrain))
erro_mes = rsme(respostas.QUANTIDADE, respostas.LAST_MONTH)

In [68]:
erro_preds_retrain, erro_media_dos_anos

(4.0871165332366743, 4.789359042926411)

## Comparação entre erros

In [93]:
def plot_erros(erros, nomes):
        
    trace1 = go.Scatter(
        x=nomes,
        y=erros,
        name='Erros das previsões 2018',
        text=np.round(erros, 2),
#         textposition = 'auto',
        marker=dict(
            sizemode = 'diameter',
            sizeref = 2,
            size = 30,
            color=erros,
            colorscale='Portland',
            showscale=False,
            reversescale = True
            ),
        mode='markers',
    )

    data = [trace1]

    layout = dict(title='Erros das previsões 2018', showlegend=False)

    fig = go.Figure(data=data, layout=layout)
    iplot(fig)

In [94]:
plot_erros([erro_ano_passado, erro_2_ano, erro_media_dos_anos, erro_preds_all, erro_preds_retrain], ['Ano passado', '2 anos atrás', 'Média dos anos', 'Previsão no começo do ano', 'Previsão de mês em mês'])

In [41]:
1 - erro_preds_retrain / erro_media_dos_anos, 1 - erro_preds_retrain / erro_ano_passado

(0.14478925751540517, 0.28343456800395328)

#### Ouve uma melhoria aproximada de 28% na predições se comparado ao método antigo usado, e de aproximadamente 14% se comparado com a comparação entre a média do histórico de vendas

In [136]:
def plot_erros(df, produto):
    
#     total = df[df.DESCRICAO == produto][respostas.TAMANHO == tamanho].groupby(['MES', 'ANO', 'DESCRICAO', 'PRECO_PRODUTO']).sum()
    total = df[df.DESCRICAO == produto].groupby(['MES', 'ANO', 'DESCRICAO', 'PRECO_PRODUTO']).sum()

    meses = ['Jan/18', 'Fev/18', 'Março/18', 'Abril/18', 'Maio/18']
    
    trace1 = go.Scatter(
        x=meses,
        y=total.QUANTIDADE,
        name='Real',
        marker=dict(
            color='black'
            ),
        opacity=0.6
    )
    
    trace2 = go.Scatter(
        x=meses,
        y=np.round((total.preds_retrain + total.preds_all)/2),
        name='Machine Learning - Pred',
        marker=dict(
            color='green'
            ),
        opacity=0.6,
        mode='line',
        line = dict(dash = 'dot')
    )
    
    trace3 = go.Scatter(
        x=meses,
        y=np.round(total.LAST_YEAR),
        name='Historical data',
        marker=dict(
            color='red'
            ),
        opacity=0.6,
        line = dict(dash = 'dot')
    )

    data = [trace1, trace2, trace3]

    layout = dict(title='Erros das previsões por produto', showlegend=True)

    fig = go.Figure(data=data, layout=layout)
    iplot(fig)

In [137]:
plot_erros(respostas, 'CORSARIO SUPLEX')
# interact(plot_erros, df=fixed(respostas), produto=pd.unique(respostas.DESCRICAO))

In [291]:
soma = respostas.groupby(['DESCRICAO', 'MES', 'TAMANHO', 'PRECO_PRODUTO']).sum().reset_index()

In [292]:
soma['LAST_YEAR'] = soma.LAST_YEAR.apply(np.round)
soma['preds_retrain'] = soma.preds_retrain.apply(np.round)

In [293]:
soma['DIFERENCA_LAST_YEAR'] = soma['LAST_YEAR'] - soma['QUANTIDADE']
soma['DIFERENCA_PREDS'] = soma['preds_retrain'] - soma['QUANTIDADE']

In [297]:
soma['VALOR_PREDS'] = soma.PRECO_PRODUTO * soma.DIFERENCA_PREDS
soma['VALOR_LAST_YEAR'] = soma.PRECO_PRODUTO * soma.DIFERENCA_LAST_YEAR

In [304]:
em_estoque_pred = soma['VALOR_PREDS'][soma.VALOR_PREDS > 0 ].sum()
em_estoque_ultimo = soma['VALOR_LAST_YEAR'][soma.VALOR_LAST_YEAR > 0 ].sum()
deixou_de_vender_pred = soma['VALOR_PREDS'][soma.VALOR_PREDS < 0 ].sum()
deixou_de_vender_ultimo = soma['VALOR_LAST_YEAR'][soma.VALOR_LAST_YEAR < 0 ].sum()

In [330]:
trace1 = go.Bar(
        y=[em_estoque_ultimo, em_estoque_pred],
        x=['Ultimo Ano', 'Previsão'],
        name='Real',
        marker=dict(
            color='green'
            ),
    )

trace2 = go.Bar(
        y=[deixou_de_vender_ultimo, deixou_de_vender_pred],
        x=['Ultimo Ano', 'Previsão'],
        name='Real',
        marker=dict(
            color='red'
            ),
    )

In [332]:
fig = go.Figure(data=[trace1, trace2], layout={'title':"Melhoria Financeira da previsão"})
iplot(fig)

In [339]:
print("R$", deixou_de_vender_pred - deixou_de_vender_ultimo, "a menos parado em estoque")
print("Economia de ", np.round(100*(1 - deixou_de_vender_pred / deixou_de_vender_ultimo),2), "%")

R$ 25303.5 a menos parado em estoque
Economia de  20.12 %


In [344]:
print("R$", em_estoque_pred - em_estoque_ultimo, " a mais de venda devido a falta de estoque")
print("Melhoria de ", np.round(100*(1 - em_estoque_pred / em_estoque_ultimo),2), "%")

R$ -21941.5  a mais de venda devido a falta de estoque
Melhoria de  17.1 %
